In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import split, when, col, lit, trim

In [ ]:
# create spark dataframe
df = spark.read.option("header",True).csv("play_by_play_sample.csv")

In [3]:
# calculate how much time is left in seconds in the quarter
# split pctimestring column into minutes and seconds and add them as columns to the dataframe
df = df.withColumn("minutes", split(df["pctimestring"], ":").getItem(0).cast("int"))
df = df.withColumn("seconds", split(df["pctimestring"], ":").getItem(1).cast("int"))

# calculate the total seconds
df = df.withColumn("seconds_left_in_game", df["minutes"] * 60 + df["seconds"])
# df.show(30,False)

In [4]:
# only select plays in the last 5 mins of the game, and where a point is scored
df = df.where((df.period == 4) & (df.seconds_left_in_game < 300) & (df.scoremargin.isNotNull()) & (df.eventmsgtype.isin (1,3)))
# only select needed columns
cols = ["eventmsgtype","eventmsgactiontype","period","seconds_left_in_game","homedescription","visitordescription", "player1_id", "player1_name", "player1_team_abbreviation", "score","scoremargin"]
# df.select(cols).show(20,False)

In [ ]:
# define function for 3PT regex
# must match exactly " 3PT " as if a player has 3 total points it will read "... (3 PTS) ..."
def regex_3PT(description_col):
    return trim(col(description_col)).contains(" 3PT ")

# add points scored column
df = df.withColumn(
    "points_scored",
    when(col("eventmsgtype") == 3, lit(1))
    .when(
        regex_3PT("homedescription") | regex_3PT("visitordescription"), lit(3)
    )
    .otherwise(lit(2))
)

df.show(10,False)

In [ ]:
# cast points scored to int
df = df.withColumn("points_scored", df.points_scored.cast('int'))
# group players by points scored, label column clutch_points
clutch_players = df.groupBy("player1_name").sum("points_scored").select(col("player1_name"), col("sum(points_scored)").alias("clutch_points"))
# show and sort by descending order
clutch_players.orderBy("clutch_points", ascending = False).show()

In [ ]:
# queries for eventmsgtype and eventmsgactiontype mapping
df.select("eventmsgtype","eventmsgactiontype").distinct().show()
df.where((df.eventmsgtype == 1) & (df.eventmsgactiontype == 8)).show(50,False)

## eventmsgtype and eventmsgactiontype mapping
eventmsgtype - eventmsgactiontype ----- PLAY \
------ 1 ----------------- 1 --------------- jump shot (2 or 3 pts) \
------ 1 ----------------- 2 --------------- running jump shot \
------ 1 ----------------- 3 --------------- hook shot \
------ 1 ----------------- 4 --------------- tip shot \
------ 1 ----------------- 5 --------------- layup \
------ 1 ----------------- 6 --------------- driving layup \
------ 1 ----------------- 7 --------------- dunk \
------ 1 ----------------- 8 --------------- slam dunk \
------ 3 ----------------- 10 -------------- 1 of 1 FT \
------ 3 ----------------- 11 -------------- 1 of 2 FT \
------ 3 ----------------- 12 -------------- 2 of 2 FT \
------ 3 ----------------- 13 -------------- 1 of 3 FT \
------ 3 ----------------- 14 -------------- 2 of 3 FT \
------ 3 ----------------- 15 -------------- 3 of 3 FT \
------ 3 ----------------- 16 -------------- technical FT

In [70]:
# create temp view to query dataframe using sql
df.createOrReplaceTempView("play_by_play")
# query data using SQL
query = ("SELECT eventmsgtype, eventmsgactiontype, period, seconds_left_in_game"
         ", player1_name, player1_team_abbreviation "
         "FROM play_by_play WHERE period = 4 AND seconds_left_in_game < 300 limit 10;")
result = spark.sql(query)
result.show()